# Train the RPS model

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kjy5/cv-rock-paper-scissors/blob/main/scripts/train.ipynb)

# TEMP: resources:
- For transfer learning see [transfer learning](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)

- For saving/loading: [save load run](https://pytorch.org/tutorials/beginner/basics/saveloadrun_tutorial.html)

## Setup
### 1. Download data
Only need to run this once

In [ ]:
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=11IkCeaEsjysSaWgMEI3SwkSzJ1Tmxz1i&confirm=t' -O data.zip
!unzip data.zip
!rm -rf data.zip

### 2. Import libraries

In [1]:
import torch
from torchvision import transforms, datasets
from torchvision.models import resnet18, ResNet18_Weights
import cv2 as cv
from PIL import Image
import os
from collections import deque
import random
import numpy as np


### 3. Set up device

In [2]:
d = torch.device("cpu")
# Use a CUDA GPU if possible (Apple Silicon MPS backend technically works, but is confusingly slow)
if torch.cuda.is_available():
    d = torch.device("cuda:0")
d

device(type='cpu')

## Loading data and model
### Define constants

In [3]:
DATA_DIR = "data"
CLASSES = ["rock", "paper", "scissors", "clutter"]
VIDEOS_PER_CLASS = 2

TRAIN_PHASE = "train"
VAL_PHASE = "val"

CLUTTER_IMAGE_PREFIX = "test_"
CLUTTER_COUNT = 10000

NUM_FRAMES = 120
TRAIN_VAL_SPLIT = int(NUM_FRAMES * 0.8)


### Define transformers

In [17]:
preprocess = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Resize(224),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)
transformers = {
    TRAIN_PHASE: transforms.Compose(
        [
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    ),
    VAL_PHASE: transforms.Compose(
        [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    ),
}


### Load data

In [21]:
# Define dataloaders
dataloaders = {
    TRAIN_PHASE: {},
    VAL_PHASE: {},
}


#### Clutter data

In [22]:
# Load 240 random clutter images
clutter_frame_ids = random.sample(range(CLUTTER_COUNT), NUM_FRAMES * 2)
FRAME_SPLIT = int(TRAIN_VAL_SPLIT * 2)

for phase, transformer in transformers.items():
    # Configure image cache
    data = deque()
    ids = (
        clutter_frame_ids[:FRAME_SPLIT]
        if phase == TRAIN_PHASE
        else clutter_frame_ids[FRAME_SPLIT:]
    )

    # Load and preprocess images
    for img_id in ids:
        cur_image = Image.open(
            os.path.join(DATA_DIR, "clutter", f"{CLUTTER_IMAGE_PREFIX}{img_id}.JPEG")
        )
        # Ensure using RGB
        if cur_image.mode != "RGB":
            cur_image = cur_image.convert("RGB")
        data.append(transformer(cur_image))
        cur_image.close()

    # Save to dataloader
    dataloaders[phase]["clutter"] = torch.utils.data.DataLoader(
        data, batch_size=4, shuffle=True, num_workers=4
    )
# Cleanup
del data, ids, clutter_frame_ids


#### RPS data (videos)

In [23]:
# Loop through each RPS class
for rps_class in CLASSES[:-1]:
    dir = os.path.join(DATA_DIR, rps_class)

    # Make frame cache
    frames = deque()

    # Loop through each video
    with os.scandir(dir) as it:
        for entry in it:
            if entry.name.endswith(".mp4") and entry.is_file():
                # Load video
                cap = cv.VideoCapture(os.path.join(dir, entry.name))

                # Load frames
                while True:
                    ret, frame = cap.read()
                    if ret:
                        # Switch channels
                        frame = frame[:, :, [2, 1, 0]]
                        frames.append(Image.fromarray(frame))
                    else:
                        break
                cap.release()

    # Shuffle frames and convert to array
    random.shuffle(frames)
    frames = list(frames)

    # Split into train and val and save to dataloader
    for phase, transformer in transformers.items():
        frame_segment = (
            frames[:TRAIN_VAL_SPLIT] if phase == TRAIN_PHASE else frames[TRAIN_VAL_SPLIT:]
        )
        dataloaders[phase][rps_class] = torch.utils.data.DataLoader(
            [transformer(img) for img in frame_segment],
            batch_size=4,
            shuffle=True,
            num_workers=4,
        )


### Load Model

In [ ]:
# Load in a pretrained model
model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
model.to(d)

In [ ]:
# Load presaved model
model = torch.load("../model.pth")
model.to(d)

# Tasks:
Read whole training videos into memory

Read clutter dataset into memory

## Training


### Load in training data
#### 1. Rock

In [ ]:
video_file = cv.VideoCapture("../data/rock/rock.mp4")

# Count frames
num_frames = 0
while True:
    ret, frame = video_file.read()
    if not ret:
        break
    num_frames += 1

frames = torch.empty(num_frames, 3, 224, 224)
frames_index = 0
video_file = cv.VideoCapture("../data/rock/rock.mp4")
while True:
    ret, frame = video_file.read()
    if not ret:
        break

    # Rearrange the channels
    image = frame[:, :, [2, 1, 0]]

    # Run preprocess
    image_tensor = preprocess(image).to(d)

    # Append to processed frames
    frames[frames_index] = image_tensor

print(f'Loaded {num_frames} frames')

In [ ]:
# Save model
torch.save(model, "../model.pth")